In [1]:
# %pip install -q -r requirements.txt

In [2]:
import sys
from pathlib import Path
sys.path.append(str(Path.cwd().parent))

In [3]:
from typing import Iterator
import os
from itertools import islice
from dotenv import load_dotenv

In [4]:
from chainweb.chaindb import ProvideChainwebDb
from chainweb.primitives import read_hash
from chainweb.header import Header
from chainweb.payload import BlockPayload
from chainweb.primitives import ChainId

In [5]:
%load_ext autoreload
%autoreload 2

In [6]:
load_dotenv()

True

# Chainweb DB Tests

In [7]:
dbPath = os.getenv('CHAINWEB_DB_PATH')
network = os.getenv('CHAINWEB_NETWORK')

In [8]:
# Tests
from chainweb.payload import get_meta

with ProvideChainwebDb(dbPath) as db:
    for k,v in db.headers[0]:
        assert k == v.height
        assert v == db.headers[0].get(k, v.hash)
        p = db.payload_data_with_outputs.get(k, v.payload_hash)
        assert p.payload_hash == v.payload_hash
        if len(p.transactions) > 0:
            tin = p.transactions[0][0]
            tout = p.transactions[0][1]
            cmd = tin['cmd']
            if cmd["networkId"] is not None:
                assert cmd["networkId"] == network
            meta = get_meta(tin)
            if meta['creationTime'] > 0:
                assert int(meta["chainId"]) == v.chain_id
                assert meta["creationTime"] < v.creation_time

In [9]:
with ProvideChainwebDb(dbPath) as db:
    h1_hex = list(islice(db.headers[0],1,2))[0][1].hash.hex()

h1 = read_hash(h1_hex)

with ProvideChainwebDb(dbPath) as db:
    h1_ = db.headers[0].get(1,h1)
    assert h1_.hash == h1
    assert h1_.height == 1

In [10]:
# This assumes that payload iteration is ordered by height.
# Due to a bug in the new chainweb rocksdb backend this is not the case for
# databases with more than 256 block heights.
#
# It can be used as a test case for proper ordering of payloads.
#
def block_iter(db, chain: ChainId) -> Iterator[tuple[int, Header, BlockPayload]]:
    hit = iter(db.headers[chain])
    pit = iter(db.payloads)
    for h, hdr in hit:
        while True:
            ph, p = next(pit)
            if ph < h:
                continue
            if ph > h:
                raise Exception(f"Payload {ph} not found for header {h}")
            if p.payload_hash == hdr.payload_hash:
                yield (h, hdr, p)
                break

with ProvideChainwebDb(dbPath) as db:
    for h, hdr, p in islice(block_iter(db, chain = ChainId(0)), 0, 5):
        print(h, hdr, p)


0 Header(headerbytes=b'\x00\x00\x00\x00\x00\x00\x00\x00\x08}e\xa9\xe4\x8d\x05\x00\xd3\xfc\x00\xaf\x01\xde\xb0\x94\x00\xc2\x00\x93\xb5\t \xc0PP\x1e\xe9\x8f\x85\xe2\xb92|\xe3_\xba\x9b`L\x03\x00\x05\x00\x00\x00\xe3\x94\x8e\xed$D\xbd\xac\xaa\t\xbbt\niD\xb2\xa7\x1a\xc8\xc8\xf6\xfb\x83\xdb\x9b\x82M\xb5o%c*\n\x00\x00\x00\xe4\xb6H\xf3\xda\x83y\xa3J\x16\xbb\x80\x87\x91\x96O\xb7\xf5\xf1\x1e\xc8i\xe808\x93X\x84\x93\xd2w\x0c\x0f\x00\x00\x00\xcf\xe1t\x80\x8c\x1aeeY\x83\x99{\xb4\xb9\xb7\xd7\x8a\xb7.\xd0\xe3\xba0\xc4lZ0\xf8\xdfUl^\xa02\xfe}\xc6\x85\x03!Y\xc0\x04n\xdd\xcdS\x1dr3\xdc\x80\xcf\x0f#\x84G\x1bG\xac\xc5\xa7\x00\x00C<U\x1cVy\x82\x8e\x0fa\xdd\xd0x\x06a\xc4\xfe\xb5\x86\xc5g #\xf0\xe0\x10}\x87\xe4\xf7<3\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x08}e\xa9\xe4\x8d\x05\x00\x00\x00\x00\x00\x00\x00\x00\x00\xc2\x85\xb1~\xece+2\xab\xf5\x86\xd5ON\xda*\xa2\

# Debug block validation failure

In [11]:
import json
import dictdiffer
from chainweb.primitives import BlockHeight

failure_file = os.getenv('CHAINWEB_FAILURE_FILE')

In [12]:
# Utils

def trim_logs(x):
    if 'hash' in x['logs']:
        x['logs'] = x['logs']['hash']
    return x

def get_failure_details(x):
    return x['contents']['details'][0]


In [13]:
# Read Failure JSON File
with open(failure_file, "r") as fp:
    r = json.load(fp)

failure = get_failure_details(r)
expected_output_hash = failure['mismatch']['expected']
actual_coinbase = trim_logs(failure['outputs']['coinbase'])
actual_tx_outputs = list(map(lambda x: trim_logs(x['result']), failure['outputs']['txs']))

In [14]:
failure['outputs']['txs'][0]['tx']

{'hash': 'ICNCNiAXmvGTMqiYHo7QhT46RqOT26U1-i22YWnpCpo',
 'sigs': [{'sig': '2d29616ec7164a67fb7b51a137eaa1ce65f5ff07e551d93a4c9d699db4b3ffcc6db0b7ea334032a79248890d2d4fc4f0e76a8cf21bf5706cc53cb00e46893a06'},
  {'sig': 'e311d9e5d5656a221d135f231c9bb0122ce94561a4b06a95aaeacf6f30a0920502c00a48a887fb64fe756cab6226d8ac2e2017afdd36b9b6726f6e718ca00009'}],
 'cmd': {'networkId': 'development',
  'payload': {'exec': {'data': {'admin-keyset': ['368820f80c324bbc7c2b0610688a7da43e39f91d118732671cd9c7500ff43cca']},
    'code': '(namespace "ac7d37292-adcb-4d79-b815-0653257a283c") \n (interface iface\n  (defun f:bool ()))\n\n(module evmodule G\n\n  (defcap G () true)\n\n  (implements iface)\n\n  (defun f:bool () true)\n\n  (defcap EVENT (mod:module{iface})\n    @event true)\n\n  (defun emit(mod:module{iface})\n    (emit-event (EVENT mod)))\n\n  (defun emit-transfer(sender:string receiver:string amount:decimal)\n    (emit-event (coin.TRANSFER sender receiver amount))))\n \n\n       (ac7d37292-adcb-4d79

In [15]:
actual_tx_outputs

[{'gas': 123309,
  'result': {'status': 'success', 'data': True},
  'reqKey': 'ICNCNiAXmvGTMqiYHo7QhT46RqOT26U1-i22YWnpCpo',
  'logs': 'J4IUfJLZLxSV4eoGm08ih3fSmGVp2HXqZIFsDq8AMZg',
  'events': [{'params': ['test-sender',
     'k:f89ef46927f506c70b6a58fd322450a936311dc6ac91f4ec3d8ef949608dbf1f',
     1.23309],
    'name': 'TRANSFER',
    'module': {'namespace': None, 'name': 'coin'},
    'moduleHash': 'wOTjNC3gtOAjqgCY8S9hQ-LBiwcPUE7j4iBDE0TmdJo'},
   {'params': [{'refSpec': [{'namespace': 'ac7d37292-adcb-4d79-b815-0653257a283c',
        'name': 'iface'}],
      'refName': {'namespace': 'ac7d37292-adcb-4d79-b815-0653257a283c',
       'name': 'evmodule'}}],
    'name': 'EVENT',
    'module': {'namespace': 'ac7d37292-adcb-4d79-b815-0653257a283c',
     'name': 'evmodule'},
    'moduleHash': 'Qk5FPtZpUYpQaJjNFUx4xQDzyGPPuydOls0Y-8UVqo0'}],
  'metaData': None,
  'continuation': None,
  'txId': 179}]

In [16]:
# Get expected outputs form the DB
with ProvideChainwebDb(dbPath) as db:
    e = db.outputs.get(BlockHeight(152), read_hash(expected_output_hash))
expected_outputs = e.outputs
expected_coinbase = e.coinbase

# display(expected_coinbase)
# display(expected_outputs)

In [17]:
expected_outputs

[{'gas': 123310,
  'result': {'status': 'success', 'data': True},
  'reqKey': 'ICNCNiAXmvGTMqiYHo7QhT46RqOT26U1-i22YWnpCpo',
  'logs': '4aPWv4UvFH3MENv_bHnmMF-vEEugvXzKsNBW79LMfsA',
  'events': [{'params': ['test-sender',
     'k:f89ef46927f506c70b6a58fd322450a936311dc6ac91f4ec3d8ef949608dbf1f',
     1.2331],
    'name': 'TRANSFER',
    'module': {'namespace': None, 'name': 'coin'},
    'moduleHash': 'wOTjNC3gtOAjqgCY8S9hQ-LBiwcPUE7j4iBDE0TmdJo'},
   {'params': [{'refSpec': [{'namespace': 'ac7d37292-adcb-4d79-b815-0653257a283c',
        'name': 'iface'}],
      'refName': {'namespace': 'ac7d37292-adcb-4d79-b815-0653257a283c',
       'name': 'evmodule'}}],
    'name': 'EVENT',
    'module': {'namespace': 'ac7d37292-adcb-4d79-b815-0653257a283c',
     'name': 'evmodule'},
    'moduleHash': 'Qk5FPtZpUYpQaJjNFUx4xQDzyGPPuydOls0Y-8UVqo0'}],
  'metaData': None,
  'continuation': None,
  'txId': 179}]

In [18]:
# Check for differences in coinbase output
list(dictdiffer.diff(expected_coinbase, actual_coinbase))

[]

In [19]:
# Check for differences in tx outputs
list(dictdiffer.diff(expected_outputs[0], actual_tx_outputs[0]))

[('change', 'gas', (123310, 123309)),
 ('change',
  'logs',
  ('4aPWv4UvFH3MENv_bHnmMF-vEEugvXzKsNBW79LMfsA',
   'J4IUfJLZLxSV4eoGm08ih3fSmGVp2HXqZIFsDq8AMZg')),
 ('change', ['events', 0, 'params', 2], (1.2331, 1.23309))]